#### <h1><center>Part 4: Summarization and Q/A </center></h1>

In part 4, we use the result found in part 1 and new GPT capability to extract pros and cons of the product and generate recommendation based on the reviews.


## 1. Data Preparation 

#### Import of different libraries that will be used in this part of the project.

In [76]:
import pandas as pd
import re
import math
from collections import Counter
import matplotlib.pyplot as plt
import pygal
from IPython.display import SVG, display
import numpy as np
import openai

In [77]:
openai.api_key = "*****"
model_engine = "text-davinci-003"

In [78]:
dat = pd.read_csv('asin_sample_GPT_Rev.csv')

In [79]:
data = dat.drop(['Unnamed: 0','reviewerID','asin','summary','unixReviewTime','reviewText_deep_clean'], axis=1)

In [80]:
data_pos = data[data['Sentiment_GPT_Score']==1]
data_pos = data_pos[data_pos['vote'] > 0]
data_neg = data[data['Sentiment_GPT_Score']== -1]
data_neg = data_neg[data_neg['vote'] > 0]

#### Find the most voted positive and negative review

In [81]:
# find the index of the row with the highest number of votes
max_votes_index = data_neg['vote'].idxmax()

# access the review with the highest number of votes
review_with_max_votes = data_neg.loc[max_votes_index, 'reviewText']
max_votes = data_neg.loc[max_votes_index, 'vote']

# print the review with the highest number of votes
print("Below is the most voted Negative review with", max_votes, "votes:\n\n",review_with_max_votes)

Below is the most voted Negative review with 152 votes:

 Our filter cracked and flooded our condo and the two floors below us too. Maybe even three floors down but after seeing the two below us I didn't have the heart to look any further. We weren't home and got a call from our neighbor at 1 am that our hallway was flooded and it all came from this water filter. It developed a huge crack along the bottom and ran continuously like a faucet into our fridge and out onto our floor and beyond. Save yourself thousands of dollars worth of damage and buy the more expensive name brand version of this filter.


In [82]:
# find the index of the row with the highest number of votes
max_votes_index = data_pos['vote'].idxmax()

# access the review with the highest number of votes
review_with_max_votes = data_pos.loc[max_votes_index, 'reviewText']
max_votes = data_pos.loc[max_votes_index, 'vote']

# print the review with the highest number of votes
print("Below is the most voted Negative review with", max_votes, "votes:\n\n",review_with_max_votes)

Below is the most voted Negative review with 124 votes:

 Have an old GE refrigerator that had a GE GWF filter in it. Purchased this filter since it had the same NSF ratings as the GE replacement filter but costs significantly less money. Installation was simple and took less than a minute. Fit was fine and there were no leaks. Flushed the filter as instructed and immediately noticed water flow was much stronger than with the old filter. Water tastes fine and there is no noticeable difference from the genuine GE filter. I am very satisfied with this product.


## 2. Pros and Cons

In [83]:
# Get the positive and negative reviews as separate lists
positive_reviews = data_pos['reviewText_clean'].tolist()
negative_reviews = data_neg['reviewText_clean'].tolist()

# Define the maximum prompt length
MAX_PROMPT_LENGTH = 1799

# Define the GPT-3 prompt to summarize positive features
positive_prompt = "Generate the main pros of the product based on the following positive reviews:\n"
for review in positive_reviews:
    prompt_length = len(positive_prompt) + len("- " + review + "\n")
    if prompt_length <= MAX_PROMPT_LENGTH:
        positive_prompt += "- " + review + "\n"
    else:
        break

# Define the GPT-3 prompt to summarize negative features
negative_prompt = "Generate the main cons of the product based on the following negative reviews:\n"
for review in negative_reviews:
    prompt_length = len(negative_prompt) + len("- " + review + "\n")
    if prompt_length <= MAX_PROMPT_LENGTH:
        negative_prompt += "- " + review + "\n"
    else:
        break

# Generate text using GPT-3 for positive features
positive_response = openai.Completion.create(
  engine= model_engine,
  prompt=positive_prompt[:MAX_PROMPT_LENGTH],
  max_tokens=200,
  n=1,
  stop=None,
  temperature=0.5
)

# Generate text using GPT-3 for negative features
negative_response = openai.Completion.create(
  engine= model_engine,
  prompt=negative_prompt[:MAX_PROMPT_LENGTH],
  max_tokens=200,
  n=1,
  stop=None,
  temperature=0.5
)

# Print the generated text
print(positive_response.choices[0].text)
print(negative_response.choices[0].text)


Main Pros: 
- Easy installation with clear instructions
- Significantly lower cost option with no apparent decrease in quality
- Excellent waterflow and clean taste
- Saves money compared to original manufacturers filter
- Never leaks
- Fast shipping

Main cons of the product:
- Poor quality filter that does not fit as stated
- Causes water to come out with a yellow color and weird taste
- Can cause damage to filter assembly resulting in a small trickle of water
- May be faulty or defective resulting in having to return the product


## 3. Generate recommendations

### We ask GPT model this question: What is your opinion about this product based on positive reviews?

In [93]:
# define the prompt to ask for GPT-3's opinion about a product based on reviews
prompt = "What is your opinion about this GE MWF SmartWater Compatible Water Filter Cartridge based on the following positive reviews:\n"
for review in positive_reviews:
    prompt_length = len(prompt) + len("- " + review + "\n")
    if prompt_length <= MAX_PROMPT_LENGTH:
        prompt += "- " + review + "\n"
    else:
        break
        
# use the OpenAI API to generate text based on the prompt
response = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=250,
    n=1,
    stop=None,
    temperature=0.5,
)

# get the generated text from the API response
generated_text = response.choices[0].text

# print the generated text
print(generated_text)



Based on the positive reviews, it appears that the GE MWF SmartWater Compatible Water Filter Cartridge is a great product. It is easy to install, provides good water flow, clean taste, and is cost effective. Customers are also pleased with the fast shipping and the fact that it does not leak. Overall, it is a great product and worth considering.


### We ask GPT model this question: What is your opinion about this product based on negative reviews?

In [94]:
# define the prompt to ask for GPT-3's opinion about a product based on reviews
prompt = "What is your opinion about this GE MWF SmartWater Compatible Water Filter Cartridge based on the following negative reviews:\n"
# prompt = "Please suggest some recommendation to improve the quality/service of this GE MWF SmartWater Compatible Water Filter Cartridge based on the following negative reviews:\n"
for review in negative_reviews:
    prompt_length = len(prompt) + len("- " + review + "\n")
    if prompt_length <= MAX_PROMPT_LENGTH:
        prompt += "- " + review + "\n"
    else:
        break
        
# use the OpenAI API to generate text based on the prompt
response = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=250,
    n=1,
    stop=None,
    temperature=0.5,
)

# get the generated text from the API response
generated_text = response.choices[0].text

# print the generated text
print(generated_text)



Based on these reviews, it appears that the GE MWF SmartWater Compatible Water Filter Cartridge is not a reliable product. It appears that the filter may not fit correctly, the water may come out with a yellow color and weird taste, and the filter assembly may be damaged. It is best to avoid this product and opt for a more reliable and higher quality filter.


### We ask GPT model this request: Suggest some recommendation to improve the quality/service of this prodcut/

In [95]:
# define the prompt to ask for GPT-3's opinion about a product based on reviews
# prompt = "What is your opinion about this GE MWF SmartWater Compatible Water Filter Cartridge based on the following negative reviews:\n"
prompt = "Please suggest some recommendation to improve the quality/service of this GE MWF SmartWater Compatible Water Filter Cartridge based on the following negative reviews:\n"
for review in negative_reviews:
    prompt_length = len(prompt) + len("- " + review + "\n")
    if prompt_length <= MAX_PROMPT_LENGTH:
        prompt += "- " + review + "\n"
    else:
        break
        
# use the OpenAI API to generate text based on the prompt
response = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=250,
    n=1,
    stop=None,
    temperature=0.5,
)

# get the generated text from the API response
generated_text = response.choices[0].text

# print the generated text
print(generated_text)



1. Improve the quality control of the filters to ensure that each filter is free of any defects before packaging and shipping. 
2. Ensure that the filters are accurately described and fit properly for the intended model.
3. Provide a warranty or guarantee for the filters, so that customers can be assured of the quality of the product.
4. Offer a customer service helpline or online chat service to address any issues or concerns quickly and efficiently.
5. Provide clear instructions and diagrams on how to properly install and replace the filter.
6. Offer a satisfaction guarantee or money back policy, so customers can be sure of their purchase.
7. Increase the durability of the filters, so that they last longer and need to be replaced less often.


### -----------------------------------------------------------------------------------------------------------------------------------------------------
### End of project